![](https://www.zorgkaartnederland.nl/bundles/zkncontent/images/elements/logo_zkn.svg)

In [1]:
import zorgkaart

In [2]:
import folium
import numpy as np
import os
import pandas as pd
import seaborn as sns
from ipywidgets import *
from folium.plugins import MarkerCluster
from ipywidgets import interact

#### Organisatietypen

In [3]:
types = zorgkaart.get_types()
types.sample(n=10)

,organisatietype,aantal
organisatietype_id,,
pijncentrum,Pijncentrum,54
jeugdgezondheidszorg,Jeugdgezondheidszorg,200
instelling-voor-visueel-gehandicapten,Instelling voor visueel gehandicapten,20
fysiotherapiepraktijk,Fysiotherapiepraktijk,7942
woonvoorziening-voor-visueel-gehandicapten,Woonvoorziening voor visueel gehandicapten,31
verpleeghuis-en-verzorgingshuis,Verpleeghuis en verzorgingshuis,2432
logeerhuis-voor-verstandelijk-gehandicapten,Logeerhuis voor verstandelijk gehandicapten,96
cardiologiecentrum,Cardiologiecentrum,33
epilepsiekliniek,Epilepsiekliniek,17


## Dialysecentrum: info

In [4]:
types[(types['aantal'] >= 100) & (types['aantal'] <= 200)].sort_values(by='aantal')

,organisatietype,aantal
organisatietype_id,,
dialysecentrum,Dialysecentrum,103
huisartsenpost,Huisartsenpost,108
zorgaanbieder-overig,Zorgaanbieder overig,132
esthetisch-medisch-centrum,Esthetisch-medisch centrum,152
hospice,Hospice,156
tandprothetische-praktijk,Tandprothetische praktijk,192
jeugdgezondheidszorg,Jeugdgezondheidszorg,200


In [5]:
df = zorgkaart.get_info('Dialysecentrum')

100%|██████████| 6/6 [00:17<00:00,  2.70s/it]


In [6]:
m = folium.Map(location = [52.0912374, 5.14905414], zoom_start = 5)

marker_cluster = MarkerCluster().add_to(m)

for row in df.iterrows():
    row_values = row[1]
    location = (row_values.latitude, row_values.longitude)
    instelling = row_values.zorginstelling
    plaats = row_values.plaats
    rating = '{} ({} waarderingen)'.format(row_values.beoordeling, row_values.waarderingen)
    popup = '<b>{}</b><br><i>{}</i><br>{}'.format(instelling, plaats, rating)
    icon=folium.Icon(color='blue', icon='info-sign')
    marker = folium.Marker(location = location, popup = popup, icon=icon)
    marker.add_to(marker_cluster)

display(m)

## Zorghotel: details

In [7]:
types[(types['aantal'] >= 25) & (types['aantal'] <= 50)].sort_values(by='aantal')

,organisatietype,aantal
organisatietype_id,,
centrum-voor-eetstoornissen,Centrum voor eetstoornissen,25
verpleegkundig-kinderdagverblijf,Verpleegkundig kinderdagverblijf,28
belangenorganisatie,Belangenorganisatie,29
radiotherapeutisch-centrum,Radiotherapeutisch centrum,29
centrum-voor-bijzondere-tandheelkunde,Centrum voor bijzondere tandheelkunde,31
woonvoorziening-voor-visueel-gehandicapten,Woonvoorziening voor visueel gehandicapten,31
zorgkantoor,Zorgkantoor,31
cardiologiecentrum,Cardiologiecentrum,33
zorghotel,Zorghotel,35


In [8]:
organisatietype = 'Zorghotel'
zorgkaart.get_info(organisatietype)
zorgkaart.get_details(organisatietype)
df = zorgkaart.merge_datasets(organisatietype, delete=True)

100%|██████████| 35/35 [01:31<00:00,  2.29s/it]


In [9]:
def kaart(beoordeling):
    m = folium.Map(location = [52.0912374, 5.14905414], zoom_start = 9)
    for row in df.iterrows():
        row_values = row[1]
        location = (row_values.latitude, row_values.longitude)
        inst = row_values.zorginstelling
        adr1 = row_values.adres
        adr2 = row_values.postcode + '  ' + row_values.plaats
        tel = row_values.telefoon
        bw = '{} ({} waarderingen)'.format(row_values.beoordeling, row_values.waarderingen)
        web = '<a href={0} target="_blank"</a>{0}</a>'.format(row_values.website)
        popup = '<b>{}</b><br><i>{}</i><br><i>{}</i><br>{}<br>{}<br>{}'.format(inst, adr1, adr2, tel, web, bw)
        if np.isnan(row_values.beoordeling):
            icon=folium.Icon(color='blue', icon='question-sign')
        elif row_values.beoordeling >= beoordeling:
            icon=folium.Icon(color='green', icon='thumbs-up')
        else:
            icon=folium.Icon(color='orange', icon='thumbs-down')
        marker = folium.Marker(location = location, popup = popup, icon=icon)
        marker.add_to(m)
    return display(m)

In [10]:
interact(kaart, beoordeling = widgets.FloatSlider(value=8.5, min=0, max=10, step=0.5, description='Beoordeling'))

interactive(children=(FloatSlider(value=8.5, description='Beoordeling', max=10.0, step=0.5), Output()), _dom_c…

<function __main__.kaart(beoordeling)>

## Ziekenhuis: wachttijden

In [11]:
types[types['organisatietype'].str.endswith('huis')]

,organisatietype,aantal
organisatietype_id,,
verpleeghuis-en-verzorgingshuis,Verpleeghuis en verzorgingshuis,2432
ziekenhuis,Ziekenhuis,303


In [12]:
if not ('ziekenhuis_wachttijden.csv' in os.listdir('data') and 'ziekenhuis.csv' in os.listdir('data')):
    zorgkaart.get_info('Ziekenhuis', reasonable_rate=0) # warning: unreasonable rate (reasonable rate off)
    zorgkaart.get_details('Ziekenhuis', reasonable_rate=0) # warning: unreasonable rate (reasonable rate off)
    zorgkaart.merge_datasets('Ziekenhuis', delete=True)
    zorgkaart.get_wachttijden('Ziekenhuis', reasonable_rate=0) # warning: unreasonable rate (reasonable rate off)

In [13]:
wachttijden = pd.read_csv(os.path.join('data', 'ziekenhuis_wachttijden.csv'), index_col='id')
ziekenhuis = pd.read_csv(os.path.join('data', 'ziekenhuis.csv'), index_col='id')
df = pd.merge(ziekenhuis, wachttijden)

In [14]:
def wachttijden(specialisme, Treeknorm):
    df_specialisme = df[df.specialisme==specialisme]
    m = folium.Map(location = [52.0912374, 5.14905414], zoom_start = 8)
    for row in df_specialisme.iterrows():
        row_values = row[1]
        location = (row_values.latitude, row_values.longitude)
        inst = row_values.zorginstelling
        adr1 = row_values.adres
        adr2 = row_values.postcode + '  ' + row_values.plaats
        tel = row_values.telefoon
        bw = '{} ({} waarderingen)'.format(row_values.beoordeling, row_values.waarderingen)
        web = '<a href={0} target="_blank"</a>{0}</a>'.format(row_values.website)
        wtijd = row_values.wachttijd
        wtext = 'Wachttijd: {} weken'.format(wtijd)
        popup = '<b>{}</b><br><i>{}</i><br><i>{}</i><br>{}<br>{}<br>{}<br>{}'.format(inst, adr1, adr2, tel, web, bw, wtext)
        if wtijd <= Treeknorm:
            icon=folium.Icon(color='green', icon='info-sign')
        else:
            icon=folium.Icon(color='red', icon='info-sign')
        marker = folium.Marker(location = location, popup = popup, icon=icon)
        marker.add_to(m)
    return display(m)

In [15]:
interact(wachttijden,
         specialisme = widgets.Dropdown(options=sorted(df.specialisme.unique()), description='Specialisme'),
         Treeknorm = widgets.IntSlider(4, 0, 20, description='Treeknorm'))

interactive(children=(Dropdown(description='Specialisme', options=('Allergologie', 'Cardiologie', 'Dermatologi…

<function __main__.wachttijden(specialisme, Treeknorm)>

## Delete data folder

In [16]:
try:
    for file in os.listdir('data'):
        os.remove(os.path.join('data', file))
    os.rmdir('data')
except:
    pass

## Organisatietype
Met de functie `zorgkaart.get_types()` krijg je een overzicht van alle organisatietypen waarvoor je met de module `zorgkaart` data kan ophalen van de websit [www.zorkaartnederland.nl](www.zorkaartnederland.nl).  

De variable `aantal` geeft het aantal zorginstellingen aan in betreffende categorie.

In [17]:
organisatietypen = zorgkaart.get_types()
organisatietypen.head(30)

,organisatietype,aantal
organisatietype_id,,
abortuskliniek,Abortuskliniek,14
apotheek,Apotheek,1899
arbo-en-reintegratiebedrijf,Arbo- en reïntegratiebedrijf,540
artsenpraktijk-voor-alternatieve-geneeskunde,Artsenpraktijk voor alternatieve geneeskunde,419
astmacentrum,Astmacentrum,8
audiologisch-centrum,Audiologisch centrum,46
belangenorganisatie,Belangenorganisatie,29
besnijdeniscentrum,Besnijdeniscentrum,23
bevolkingsonderzoek-kanker,Bevolkingsonderzoek kanker,5


## Info

In [18]:
info = zorgkaart.get_info('Astmacentrum')
info

100%|██████████| 1/1 [00:03<00:00,  3.29s/it]


,zorginstelling,plaats,beoordeling,waarderingen,categorie,latitude,longitude,zknl_url
id,,,,,,,,
3035478,Nederlands Astmacentrum Davos,Davos Clavadel,9.2,110,Astmacentrum,0.000000,0.000000,https://www.zorgkaartnederland.nl/zorginstelli...
3058759,Center of Excellence,Amersfoort,9.0,25,Astmacentrum,52.165986,5.407762,https://www.zorgkaartnederland.nl/zorginstelli...
109628,"Merem, locatie Hilversum",Hilversum,8.8,15,Astmacentrum,52.212541,5.201926,https://www.zorgkaartnederland.nl/zorginstelli...
9848,"CIRO, Expertisecentrum voor Chronisch Orgaanfalen",Horn,8.2,10,Astmacentrum,51.219635,5.918302,https://www.zorgkaartnederland.nl/zorginstelli...
10001580,Raack - Regionaal Astma en Allergiecentrum voo...,Groesbeek,NaN,0,Organisatie,51.801576,5.894875,https://www.zorgkaartnederland.nl/zorginstelli...
3051521,Raack - Regionaal Astma en Allergiecentrum voo...,Nijmegen,NaN,0,Astmacentrum,51.817159,5.831195,https://www.zorgkaartnederland.nl/zorginstelli...
3051520,Raack - Regionaal Astma en Allergiecentrum voo...,Nijmegen,NaN,0,Astmacentrum,51.823384,5.860092,https://www.zorgkaartnederland.nl/zorginstelli...
3048242,Raack - Regionaal Astma en Allergiecentrum voo...,Groesbeek,NaN,0,Astmacentrum,51.801910,5.898534,https://www.zorgkaartnederland.nl/zorginstelli...


In [19]:
os.listdir('data')

['astmacentrum_info.csv']

## Details

In [20]:
details = zorgkaart.get_details('Astmacentrum')
details

100%|██████████| 8/8 [00:20<00:00,  2.74s/it]


,zorginstelling,adres,postcode,plaats,telefoon,website,wachttijden_url
id,,,,,,,
3035478,Nederlands Astmacentrum Davos,Davos Klinikstrasse 6,7272,Davos Clavadel,"0041-814178000, 06-31002905 (secr. Nederland)",https://nadavos.nl/,NaN
3058759,Center of Excellence,Wiekslag 90,3815 GS,Amersfoort,033-3034590,http://www.coe-amersfoort.nl,NaN
109628,"Merem, locatie Hilversum",Soestdijkerstraatweg 129,1213 VX,Hilversum,035-6881411,http://www.merem.nl,NaN
9848,"CIRO, Expertisecentrum voor Chronisch Orgaanfalen",Hornerheide 1,6085 NM,Horn,0475-587587,http://www.ciro-horn.nl,https://www.zorgkaartnederland.nl/zorginstelli...
10001580,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,http://www.raack.org,NaN
3051521,Raack,Weg door Jonkerbos 100,6532 SZ,Nijmegen,024-3658708,http://www.raack.org,NaN
3051520,Raack,Geert Grooteplein Zuid 10,6525 GA,Nijmegen,024-3614415,c,NaN
3048242,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,http://www.raack.org,NaN


In [21]:
os.listdir('data')

['astmacentrum_details.csv', 'astmacentrum_info.csv']

## Samenvoegen

In [22]:
samen = zorgkaart.merge_datasets('Astmacentrum', delete=False)
samen

,zorginstelling,adres,postcode,plaats,telefoon,beoordeling,waarderingen,categorie,latitude,longitude,website,zorgkaart_url,wachttijden_url
id,,,,,,,,,,,,,
3035478,Nederlands Astmacentrum Davos,Davos Klinikstrasse 6,7272,Davos Clavadel,"0041-814178000, 06-31002905 (secr. Nederland)",9.2,110,Astmacentrum,0.000000,0.000000,https://nadavos.nl/,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3058759,Center of Excellence,Wiekslag 90,3815 GS,Amersfoort,033-3034590,9.0,25,Astmacentrum,52.165986,5.407762,http://www.coe-amersfoort.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
109628,"Merem, locatie Hilversum",Soestdijkerstraatweg 129,1213 VX,Hilversum,035-6881411,8.8,15,Astmacentrum,52.212541,5.201926,http://www.merem.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
9848,"CIRO, Expertisecentrum voor Chronisch Orgaanfalen",Hornerheide 1,6085 NM,Horn,0475-587587,8.2,10,Astmacentrum,51.219635,5.918302,http://www.ciro-horn.nl,https://www.zorgkaartnederland.nl/zorginstelli...,https://www.zorgkaartnederland.nl/zorginstelli...
10001580,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,NaN,0,Organisatie,51.801576,5.894875,http://www.raack.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3051521,Raack,Weg door Jonkerbos 100,6532 SZ,Nijmegen,024-3658708,NaN,0,Astmacentrum,51.817159,5.831195,http://www.raack.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3051520,Raack,Geert Grooteplein Zuid 10,6525 GA,Nijmegen,024-3614415,NaN,0,Astmacentrum,51.823384,5.860092,c,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3048242,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,NaN,0,Astmacentrum,51.801910,5.898534,http://www.raack.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN


In [23]:
os.listdir('data')

['astmacentrum_details.csv', 'astmacentrum.csv', 'astmacentrum_info.csv']

In [24]:
os.remove(os.path.join('data', 'astmacentrum.csv'))
os.listdir('data')

['astmacentrum_details.csv', 'astmacentrum_info.csv']

In [25]:
samen = zorgkaart.merge_datasets('Astmacentrum', delete=True)
os.listdir('data')

['astmacentrum.csv']

## Meerdere organisatietypen

In [26]:
# delete files
for file in os.listdir('data'):
    os.remove(os.path.join('data', file))

In [27]:
lijst = organisatietypen[organisatietypen.aantal <= 10]
lijst

,organisatietype,aantal
organisatietype_id,,
astmacentrum,Astmacentrum,8
bevolkingsonderzoek-kanker,Bevolkingsonderzoek kanker,5
bloedbank,Bloedbank,6
brandwondencentrum,Brandwondencentrum,3
ivf-kliniek,IVF-kliniek,8
ministerie,Ministerie,1
orthoptistenpraktijk,Orthoptistenpraktijk,4


In [28]:
for organisatietype in lijst.organisatietype:
    zorgkaart.get_info(organisatietype, 1)
    zorgkaart.get_details(organisatietype, 1)
    zorgkaart.merge_datasets(organisatietype, delete=True)

100%|██████████| 4/4 [00:05<00:00,  1.48s/it]


In [29]:
os.listdir('data')

['orthoptistenpraktijk.csv',
 'bevolkingsonderzoek_kanker.csv',
 'bloedbank.csv',
 'brandwondencentrum.csv',
 'ivf_kliniek.csv',
 'astmacentrum.csv',
 'ministerie.csv']

In [30]:
data = pd.read_csv(os.path.join('data', os.listdir('data')[0]))
for file in os.listdir('data')[1:]:
    data = data.append(pd.read_csv(os.path.join('data', file)))
data.reset_index(drop=True, inplace=True)

In [31]:
data.sample(n=10)

,id,zorginstelling,adres,postcode,plaats,telefoon,beoordeling,waarderingen,categorie,latitude,longitude,website,zorgkaart_url,wachttijden_url
27,3058759,Center of Excellence,Wiekslag 90,3815 GS,Amersfoort,033-3034590,9.0,25,Astmacentrum,52.165986,5.407762,http://www.coe-amersfoort.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
17,3021586,Brandwondencentrum Beverwijk,Vondellaan 13,1942 LE,Beverwijk,0251-265555,7.7,3,Brandwondencentrum,52.479128,4.650600,http://www.rkz.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
30,10001580,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,NaN,0,Organisatie,51.801576,5.894875,http://www.raack.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
21,3023854,Centrum Voortplantingsgeneeskunde Brabant,Hilvarenbeekseweg 60,5022 GC,Tilburg,013-5391313,7.9,6,IVF-kliniek,51.539765,5.102822,http://www.infocvb.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
12,103960,"Sanquin Bloedvoorziening, regio Noordoost",Hanzeplein 1 ingang 47,9713 GZ,Groningen,0800-5115,NaN,0,Bloedbank,53.221647,6.578099,http://www.sanquin.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
11,102170,"Sanquin Bloedvoorziening, regio Zuidwest, loca...",Plesmanlaan 1 A,2333 BZ,Leiden,071-5685000,6.3,1,Bloedbank,52.164290,4.477660,http://www.sanquin.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
7,3020673,Bevolkingsonderzoek Oost,Zutphenseweg 51,7418 AH,Deventer,088-1186200,NaN,0,Bevolkingsonderzoek kanker,52.240953,6.175964,http://www.bevolkingsonderzoekoost.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
18,3053075,"Nij Barrahûs, centrum voor vruchtbaarheid",Heerenveenseweg 99 B,8471 ZA,Wolvega,0561-610030,9.7,38,IVF-kliniek,52.885836,5.999668,http://nijbarrahus.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
32,3051520,Raack,Geert Grooteplein Zuid 10,6525 GA,Nijmegen,024-3614415,NaN,0,Astmacentrum,51.823384,5.860092,c,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
31,3051521,Raack,Weg door Jonkerbos 100,6532 SZ,Nijmegen,024-3658708,NaN,0,Astmacentrum,51.817159,5.831195,http://www.raack.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN


## Command line interface

In [32]:
# delete files
for file in os.listdir('data'):
    os.remove(os.path.join('data', file))

In [33]:
!python zorgkaart.py "Centrum voor bijzondere tandheelkunde"

100%|███████████████████████████████████████████| 31/31 [01:16<00:00,  2.11s/it]


In [34]:
os.listdir('data')

['centrum_voor_bijzondere_tandheelkunde.csv']